# Pokemon Database

## first we need to import the necessary libraries

In [1]:
import numpy as np
import pandas as pd

## put in a DataFrame all data from pokemon.csv

In [2]:
data = pd.read_csv('pokemon.csv')
data.head()

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0


## show all columns from the data

In [3]:
data.columns

Index(['abilities', 'against_bug', 'against_dark', 'against_dragon',
       'against_electric', 'against_fairy', 'against_fight', 'against_fire',
       'against_flying', 'against_ghost', 'against_grass', 'against_ground',
       'against_ice', 'against_normal', 'against_poison', 'against_psychic',
       'against_rock', 'against_steel', 'against_water', 'attack',
       'base_egg_steps', 'base_happiness', 'base_total', 'capture_rate',
       'classfication', 'defense', 'experience_growth', 'height_m', 'hp',
       'japanese_name', 'name', 'percentage_male', 'pokedex_number',
       'sp_attack', 'sp_defense', 'speed', 'type1', 'type2', 'weight_kg',
       'generation', 'is_legendary'],
      dtype='object')

## Create the PokemonDB Dimensional tables to import to PostgreSQL

### The pokemon classfication table will look like this:

| classficationID | classfication |
|-----------------|---------------|
| 1 | Abundance Pokémon |
| 2 | Acorn Pokémon |
| 3 | Alpha Pokémon |
| 4 | Angler Pokémon |
| 5 | Ant Pit Pokémon |


In [4]:
classID = pd.Series(range(1, len(set(data.classfication))+1))

classfications = pd.DataFrame(set(data.classfication), columns=['classfication'])
classfications.sort_values(by=['classfication'], inplace=True)
classfications.set_index(classID, inplace=True)
classfications.reset_index(inplace=True)
classfications = classfications.rename(columns = {'index':'classficationID'})
classfications.to_csv('data/classfications.csv', index=False)
classfications.head()

,classficationID,classfication
0,1,Abundance Pokémon
1,2,Acorn Pokémon
2,3,Alpha Pokémon
3,4,Angler Pokémon
4,5,Ant Pit Pokémon


### The pokemon type Dimtable will look like this:

| typeID | type |
|--------|------|
| 1 |	bug | 
| 2 |	dark | 
| 3 |	dragon | 
| 4 |	electric | 
| 5 |	fairy | 

In [5]:
typeID = pd.Series(range(1, len(set(data.type1))+1))

types = pd.DataFrame(set(data.type1), columns=['type'])
types.sort_values(by=['type'], inplace=True)
types.set_index(typeID, inplace=True)
types.reset_index(inplace=True)
types = types.rename(columns = {'index':'typeID'})
types.to_csv('data/types.csv', index=False)
types.head()

,typeID,type
0,1,bug
1,2,dark
2,3,dragon
3,4,electric
4,5,fairy


### The pokemon abilities Dimtable will look like this:

| abilityID | ability |
|-----------|---------|

In [6]:
abilitiess = data.abilities.str.split('\'')

abilities = []
for pokemon in abilitiess:
  for ability in pokemon:
    #print(ability, ability.istitle())
    if ability.istitle(): 
      abilities.append(ability)

abilityID = pd.Series(range(1, len(set(abilities))+1))

abilities = pd.DataFrame(set(abilities), columns=['ability'])
abilities.sort_values(by=['ability'], inplace=True)
abilities.set_index(abilityID, inplace=True)
abilities.reset_index(inplace=True)
abilities = abilities.rename(columns = {'index':'abilityID'})
abilities.to_csv('data/abilities.csv', index=False)
abilities.head()

,abilityID,ability
0,1,Adaptability
1,2,Aftermath
2,3,Air Lock
3,4,Analytic
4,5,Anger Point


In [7]:
data.columns

Index(['abilities', 'against_bug', 'against_dark', 'against_dragon',
       'against_electric', 'against_fairy', 'against_fight', 'against_fire',
       'against_flying', 'against_ghost', 'against_grass', 'against_ground',
       'against_ice', 'against_normal', 'against_poison', 'against_psychic',
       'against_rock', 'against_steel', 'against_water', 'attack',
       'base_egg_steps', 'base_happiness', 'base_total', 'capture_rate',
       'classfication', 'defense', 'experience_growth', 'height_m', 'hp',
       'japanese_name', 'name', 'percentage_male', 'pokedex_number',
       'sp_attack', 'sp_defense', 'speed', 'type1', 'type2', 'weight_kg',
       'generation', 'is_legendary'],
      dtype='object')

## The pokemonDB Fact table to import to postgreSQL
### This table will look like this:

| generation | pokedex_number | name | hp | attack | defense | sp_attack| sp_defense | speed | base_total | fk_type | fk_classfication |
|------------|----------------|------|----|--------|---------|----------|------------|-------|------------|---------|------------------|
| 1 | 1 |	Bulbasaur | 45 | 49 | 49 | 65 | 65 | 45 | 318 | grass | Seed Pokémon |
| 1 | 2 |	Ivysaur |	60 | 62 |	63 | 80 | 80 | 60 | 405 | grass | Seed Pokémon |
| 1 | 3 |	Venusaur | 80 | 100 | 123 | 122 | 120 | 80 | 625 | grass | Seed Pokémon |
| 1 | 4 |	Charmander | 39 | 52 | 43 | 60 | 50 | 65 | 309 | fire | Lizard Pokémon |
| 1 | 5 |	Charmeleon | 58 | 64 | 58 | 80 | 65 |	80 | 405 | fire | Flame Pokémon |

In [8]:
pokemons = data[['generation', 'pokedex_number', 'name','hp', 
                'attack', 'defense', 'sp_attack', 'sp_defense', 
                'speed', 'base_total', 'type1', 'classfication']]
pokemons.head()

,generation,pokedex_number,name,hp,attack,defense,sp_attack,sp_defense,speed,base_total,type1,classfication
0,1,1,Bulbasaur,45,49,49,65,65,45,318,grass,Seed Pokémon
1,1,2,Ivysaur,60,62,63,80,80,60,405,grass,Seed Pokémon
2,1,3,Venusaur,80,100,123,122,120,80,625,grass,Seed Pokémon
3,1,4,Charmander,39,52,43,60,50,65,309,fire,Lizard Pokémon
4,1,5,Charmeleon,58,64,58,80,65,80,405,fire,Flame Pokémon


### Here we create the foreign key for the type1 column


In [9]:
for i in range(len(pokemons)):
  for j in range(len(types)):
    if pokemons.iloc[i, 10] == types.iloc[j, 1]:
      pokemons.iloc[i, 10] = types.iloc[j ,0]

pokemons

c:\Users\PC\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,generation,pokedex_number,name,hp,attack,defense,sp_attack,sp_defense,speed,base_total,type1,classfication
0,1,1,Bulbasaur,45,49,49,65,65,45,318,10,Seed Pokémon
1,1,2,Ivysaur,60,62,63,80,80,60,405,10,Seed Pokémon
2,1,3,Venusaur,80,100,123,122,120,80,625,10,Seed Pokémon
3,1,4,Charmander,39,52,43,60,50,65,309,7,Lizard Pokémon
4,1,5,Charmeleon,58,64,58,80,65,80,405,7,Flame Pokémon
...,...,...,...,...,...,...,...,...,...,...,...,...
796,7,797,Celesteela,97,101,103,107,101,61,570,17,Launch Pokémon
797,7,798,Kartana,59,181,131,59,31,109,570,10,Drawn Sword Pokémon
798,7,799,Guzzlord,223,101,53,97,53,43,570,2,Junkivore Pokémon
799,7,800,Necrozma,97,107,101,127,89,79,600,15,Prism Pokémon


### Here we create the foreign key for the classfication

In [10]:
for i in range(len(pokemons)):
  for j in range(len(classfications)):
    if pokemons.iloc[i, 11] == classfications.iloc[j, 1]:
      pokemons.iloc[i, 11] = classfications.iloc[j ,0]#.copy(deep=True)

pokemons

c:\Users\PC\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,generation,pokedex_number,name,hp,attack,defense,sp_attack,sp_defense,speed,base_total,type1,classfication
0,1,1,Bulbasaur,45,49,49,65,65,45,318,10,449
1,1,2,Ivysaur,60,62,63,80,80,60,405,10,449
2,1,3,Venusaur,80,100,123,122,120,80,625,10,449
3,1,4,Charmander,39,52,43,60,50,65,309,7,299
4,1,5,Charmeleon,58,64,58,80,65,80,405,7,187
...,...,...,...,...,...,...,...,...,...,...,...,...
796,7,797,Celesteela,97,101,103,107,101,61,570,17,283
797,7,798,Kartana,59,181,131,59,31,109,570,10,149
798,7,799,Guzzlord,223,101,53,97,53,43,570,2,270
799,7,800,Necrozma,97,107,101,127,89,79,600,15,391


### Create the pokemons.csv


In [11]:
pokemons.to_csv('data/pokemons.csv', index=False)
pokemons.head()

,generation,pokedex_number,name,hp,attack,defense,sp_attack,sp_defense,speed,base_total,type1,classfication
0,1,1,Bulbasaur,45,49,49,65,65,45,318,10,449
1,1,2,Ivysaur,60,62,63,80,80,60,405,10,449
2,1,3,Venusaur,80,100,123,122,120,80,625,10,449
3,1,4,Charmander,39,52,43,60,50,65,309,7,299
4,1,5,Charmeleon,58,64,58,80,65,80,405,7,187
